# 0. 패키지 불러오기

In [1]:
!pip install catboost
!pip install category_encoders
!pip install optuna
!pip install kmodes
!pip install pycaret

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 661 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 348 kB 32.8 MB/s 
     |████████████████████████████████| 209 kB 45.2 MB/s 
     |████████████████████████████████| 81 kB 11.2 MB/s 
     |████████████████████████████████| 78 kB 8.4 MB/s 
     |████████████████████████████████| 147 kB 78.8 MB/s 
     |████████████████████████████████| 50 kB 7.1 MB/s 
     |████████████████████████████████| 112 kB 54.0 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=b18a42cec8fa45ab59e938af598e8417be2275f8f6c28ee044e6373c92c3b354
  Stored in directory: /root/.ca

In [3]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import math

from sklearn.ensemble import * # ExtraTreesRegressor, RandomForestClassifier
from sklearn.model_selection import * # KFold, cross_val_score, train_test_split
from sklearn.metrics import *  # make_scorer : MSE 대신 사용자가 정의한 손실함수를 사용하고 싶을 때
from sklearn.preprocessing import * # LabelEncoder, StandardScaler
from sklearn.svm import * # SVC, SVR
from sklearn.inspection import *
from sklearn.linear_model import * # LogisticRegression, RANSACRegressor, Ridge, Lasso, ElasticNet
from sklearn.decomposition import * # PCA

from category_encoders.ordinal import OrdinalEncoder
from catboost import CatBoostClassifier, Pool
from sklearn.cluster import KMeans
from kmodes.kmodes import KModes
import optuna
from optuna import Trial
from optuna.samplers import TPESampler
from pycaret.classification import *
import random

from lightgbm import *
import lightgbm as lgb
import xgboost as xgb
from datetime import datetime
import calendar
from collections import *

import warnings
warnings.filterwarnings(action = 'ignore') # 경고메세지를 무시하거나 숨긴다.

# 1. 데이터 확인

In [4]:
# 구글 드라이브 입력
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!ls '/content/drive/MyDrive/Colab Notebooks/공모전/건설기계 오일 상태 분류 AI 경진대회/open/'

data_info.xlsx	sample_submission.csv  test.csv  train.csv


In [6]:
path = '/content/drive/MyDrive/Colab Notebooks/공모전/건설기계 오일 상태 분류 AI 경진대회/open/'

Rdata_train = pd.read_csv(path + 'train.csv')
Rdata_test = pd.read_csv(path + 'test.csv')

print(Rdata_train.shape) # (14095, 54) -> 생각보다 데이터 수가 적다.
print(Rdata_train.size)
print(Rdata_test.shape) # (6041, 19)
print(Rdata_test.size)
Rdata_train.head()

(14095, 54)
761130
(6041, 19)
114779


,ID,COMPONENT_ARBITRARY,ANONYMOUS_1,YEAR,SAMPLE_TRANSFER_DAY,ANONYMOUS_2,AG,AL,B,BA,BE,CA,CD,CO,CR,CU,FH2O,FNOX,FOPTIMETHGLY,FOXID,FSO4,FTBN,FE,FUEL,H2O,K,LI,MG,MN,MO,NA,NI,P,PB,PQINDEX,S,SB,SI,SN,SOOTPERCENTAGE,TI,U100,U75,U50,U25,U20,U14,U6,U4,V,V100,V40,ZN,Y_LABEL
0,TRAIN_00000,COMPONENT3,1486,2011,7,200,0,3,93,0,0,3059,0.0,0,13,78,NaN,NaN,NaN,NaN,NaN,NaN,888,NaN,0.0,27.0,1,14,16,1,16,6,1951,0,8504,21370,0,427,2,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,154.0,75,0
1,TRAIN_00001,COMPONENT2,1350,2021,51,375,0,2,19,0,0,2978,0.0,0,0,31,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,0.0,NaN,0,1,0,0,1,0,572,2,19,1117,0,0,0,NaN,0,0.0,0.0,1.0,2.0,4.0,6.0,216.0,1454.0,0,NaN,44.0,652,0
2,TRAIN_00002,COMPONENT2,2415,2015,2,200,0,110,1,1,0,17,0.0,0,1,2,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,0.0,0.0,0,0,0,0,2,0,328,0,17,1334,0,0,1,NaN,0,0.0,0.0,0.0,0.0,3.0,39.0,11261.0,41081.0,0,NaN,72.6,412,1
3,TRAIN_00003,COMPONENT3,7389,2010,2,200,0,8,3,0,0,1960,0.0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,37,NaN,0.0,NaN,0,4,1,0,0,0,906,1,44,21774,2,1,1,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,133.3,7,0
4,TRAIN_00004,COMPONENT3,3954,2015,4,200,0,1,157,0,0,71,0.0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,71,NaN,0.0,0.0,0,0,0,0,2,0,309,0,217,18470,0,2,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,133.1,128,0


1. COMPONENT_ARBITRARY : 샘플 오일 관련 부품 (Component 4종, 비식별화) : 명목형
2. ANONYMOUS_1 : 무명 Feautre 1. 수치형 데이터
3. YEAR : 오일 샘플 및 진단 해 (Year)
4. SAMPLE_TRANSFER_DAY : 오일 샘플링 후 진단 기관으로 이동한 기간 (Days)
5. ANONYMOUS_2 : 무명 Feautre 2. 수치형 데이터
6. AG : 원소기호 AG - Silver 함유량
7. AL : 원소기호 AL - Aluminium 함유량
8. B : 원소기호 B - Boron 함유량
9. BA : 원소기호 BA -  Barium 함유량
10. BE : 원소기호 BE - Beryllium 함유량
11. CA : 원소기호 CA - Calcium 함유량
12. CD : 원소기호 CD - Cadmium 함유량
13. CO : 원소기호 CO - Cobolt 함유량
14. CR : 원소기호 CR - Chromium 함유량
15. CU : 원소기호 CU - Copper 함유량
16. FH2O : Water 수치(By FT-IR)
17. FNOX : NOx 수치(By FT-IR)
18. FOPTIMETHGLY : 비식별화
19. FOXID : Oxidation 수치(By FT-IR)
20. FSO4 : SO4 수치(By FT-IR)
21. FTBN : 염기성 첨가제물질 수치(By FT-IR)
22. FE : 원소기호 FE - Iron 함유량
23. FUEL : 연료 함유량
24. H2O : Water 함유량
25. K : 원소기호 K - Potassium 함유량
26. LI : 원소기호 LI - Lithium 함유량
27. MG : 원소기호 MG - Magnesium 함유량
28. MN : 원소기호 MN - Manganese 함유량
29. MO : 원소기호 MO - Molybdenum 함유량
30. NA : 원소기호 NA - Sodium 함유량
31. NI : 원소기호 NI - Nickel 함유량
32. P : 원소기호 P - Phosphorus 함유량
33. PB : 원소기호 PB - Lead 함유량
34. PQINDEX : Particle Quantifier Index
35. S : 원소기호 S - Sulphur 함유량
36. SB : 원소기호 SB - Antimony 함유량
37. SI : 원소기호 SI - Silicone 함유량
38. SN : 원소기호 SN - Tin 함유량
39. SOOTPERCENTAGE : Soot 함유량(%)
40. TI : 원소기호 TI - Titanium 함유량
41. U100 : Particle Count (100㎛ 이상 크기 입자)
42. U75 : Particle Count (75㎛ 이상 크기 입자)
43. U50 : Particle Count (50㎛ 이상 크기 입자)
44. U25 : Particle Count (25㎛ 이상 크기 입자)
45. U20 : Particle Count (20㎛ 이상 크기 입자)
46. U14 : Particle Count (14㎛ 이상 크기 입자)
47. U6 : Particle Count (6㎛ 이상 크기 입자)
48. U4 : Particle Count (4㎛ 이상 크기 입자)
49. V : 원소기호 V - Vanadium 함유량
50. V100 : Viscosity @ 100 degrees
51. V40 : Viscosity @ 40 degrees
52. ZN : 원소기호 ZN - Zinc 함유량
53. Y_LABEL : 오일 정상 여부 (0 : 정상, 1 : 이상)

### 준비

In [ ]:
Rdata_train.dtypes.value_counts()

# float로 되어있지만 사실 정수형인 변수들이 많다.
# 결측치가 있으면 정수형이 아닌 float로 변환되는 것 같다.

int64      31
float64    21
object      2
dtype: int64

In [ ]:
Rdata_train.select_dtypes('int64').apply(pd.Series.nunique, axis = 0)
# nunique : 고윳값의 개수를 알려준다.
# select_dtypes('int64') : int64인 변수들만 데이터로 불러온다.

ANONYMOUS_1            2951
YEAR                     16
SAMPLE_TRANSFER_DAY     107
ANONYMOUS_2             842
AG                        4
AL                      328
B                       504
BA                       38
BE                        9
CA                     2581
CO                       10
CR                      102
CU                      496
FE                     1281
LI                       36
MG                      554
MN                      100
MO                      298
NA                       83
NI                       57
P                      2191
PB                       73
PQINDEX                1910
S                      8076
SB                       38
SI                      514
SN                       37
TI                       71
V                        15
ZN                     1552
Y_LABEL                   2
dtype: int64

In [ ]:
Rdata_train.select_dtypes('float64').apply(pd.Series.nunique, axis = 0)
# 결측값은 제거한 채로 고윳값의 개수를 알려준다.

CD                   6
FH2O                72
FNOX                21
FOPTIMETHGLY         8
FOXID               34
FSO4                35
FTBN                99
FUEL                21
H2O                 47
K                  102
SOOTPERCENTAGE      55
U100                14
U75                 20
U50                 90
U25                195
U20                275
U14                549
U6                1766
U4                2053
V100               106
V40               1769
dtype: int64

In [ ]:
Rdata_train.select_dtypes('object').apply(pd.Series.nunique, axis = 0)

ID                     14095
COMPONENT_ARBITRARY        4
dtype: int64

In [ ]:
Rdata_train.columns

Index(['ID', 'COMPONENT_ARBITRARY', 'ANONYMOUS_1', 'YEAR',
       'SAMPLE_TRANSFER_DAY', 'ANONYMOUS_2', 'AG', 'AL', 'B', 'BA', 'BE', 'CA',
       'CD', 'CO', 'CR', 'CU', 'FH2O', 'FNOX', 'FOPTIMETHGLY', 'FOXID', 'FSO4',
       'FTBN', 'FE', 'FUEL', 'H2O', 'K', 'LI', 'MG', 'MN', 'MO', 'NA', 'NI',
       'P', 'PB', 'PQINDEX', 'S', 'SB', 'SI', 'SN', 'SOOTPERCENTAGE', 'TI',
       'U100', 'U75', 'U50', 'U25', 'U20', 'U14', 'U6', 'U4', 'V', 'V100',
       'V40', 'ZN', 'Y_LABEL'],
      dtype='object')

# 2. Teacher model : AL, BA 사용하여 Catboost(변수 20개)  
#    Student model : Catboost(변수 18개) 

## (1)  model1

new_var_set = ['FOPTIMETHGLY', 'FE', 'ZN', 'NI', 'BA', 
               'FNOX', 'CR', 'SI', 'CA', 'U75', 
               'K', 'SN', 'AL', 'V40', 'NA', 
               'FH2O', 'U4', 'ANONYMOUS_1', 'S', 'SB', 
               'FTBN', 'CU', 'B', 'PQINDEX', 'ANONYMOUS_2',
               'COMPONENT_ARBITRARY', 'YEAR']  
주 변수 -> 'COMPONENT_ARBITRARY', 'ANONYMOUS_1', 'YEAR', 'ANONYMOUS_2', 'CR', 'CU', 'FE', 'NI', 'PQINDEX', 'V40', 'ZN' -> 11개  
부 변수(결측 없음) -> 'BA', 'SI', 'CA', 'SN', 'AL', 'NA', 'S', 'SB', 'B' -> 9개  
부 별수(결측 있음) -> 'FH2O', 'FNOX', 'FOPTIMETHGLY', 'FTBN', 'K', 'U75', 'U4' -> 7개  

In [53]:
train1 = Rdata_train.copy()
test1 = Rdata_test.copy()

In [54]:
# AL과 BA만 추가하여 MODEL1을 만들자.
train2 = train1.loc[:, ['COMPONENT_ARBITRARY', 'ANONYMOUS_1', 'YEAR', 'ANONYMOUS_2', 'AG',
                                        'CO', 'CR', 'CU', 'FE', 'H2O', 'MN', 'MO', 'NI', 'PQINDEX', 'TI', 'V',
                                        'V40', 'ZN', 'Y_LABEL', 'AL', 'BA']]
test2 = test1.drop(['ID'], axis = 1)

print(train2.shape) # 변수 20개
print(test2.shape) # 변수 18개

(14095, 21)
(6041, 18)


In [55]:
# 이 작업을 수행하면 범주형 변수인 COMPONENT_ARBITRARY와 YEAR를 숫자로 바꿔준다.
# 단 OneHotEncoder가 아님, 내가 쓴건 LabelEncoder
# OneHotEncoder -> 개 : 1 0 0, 고양이 : 0 1 0, 사자 : 0 0 1
# LabelEncoder -> 개 : 0, 고양이 : 1, 사자 : 2

le1 = LabelEncoder()
le2 = LabelEncoder()

train2['COMPONENT_ARBITRARY_category'] = le1.fit_transform(train2['COMPONENT_ARBITRARY'])
train2['YEAR_category'] = le2.fit_transform(train2['YEAR'])

test2['COMPONENT_ARBITRARY_category'] = le1.transform(test2['COMPONENT_ARBITRARY'])
test2['YEAR_category'] = le2.transform(test2['YEAR'])

# 원래 범주형 변수는 제거해준다.
train3 = train2.drop(['COMPONENT_ARBITRARY', 'YEAR'], axis = 1)
test3 = test2.drop(['COMPONENT_ARBITRARY', 'YEAR'], axis = 1)

In [56]:
# 범주형 변수와 숫자형 변수를 나눠준다.
categorical_features = ['COMPONENT_ARBITRARY_category', 'YEAR_category']
numeric_features_train = ['ANONYMOUS_1', 'ANONYMOUS_2', 'AG', 'CO', 'CR', 'CU', 
                    'FE', 'H2O', 'MN', 'MO', 'NI', 'PQINDEX', 'TI', 'V', 'V40', 'ZN', 'AL', 'BA']
numeric_features_test = ['ANONYMOUS_1', 'ANONYMOUS_2', 'AG', 'CO', 'CR', 'CU', 
                    'FE', 'H2O', 'MN', 'MO', 'NI', 'PQINDEX', 'TI', 'V', 'V40', 'ZN']

print(train3.shape)
print(test3.shape)

(14095, 21)
(6041, 18)


In [57]:
X_train = train3.drop(['Y_LABEL'], axis = 1)
y_train = train3['Y_LABEL']
X_test = test3

In [58]:
# 교차 검증을 위해 train데이터를 partrain과 val로 나눠준다.

X_partrain, X_val, y_partrain, y_val = train_test_split(X_train, y_train, test_size = 0.3, random_state = 39, stratify = y_train)
print(X_partrain.shape)
print(X_val.shape)
print(y_partrain.shape)
print(y_val.shape)

(9866, 20)
(4229, 20)
(9866,)
(4229,)


In [13]:
# 교차검증 패키지인 optuna 사용을 한다.
# 이 작업은 어떤 초모수일때, 모델이 가장 좋은지 적합해준다.

def objective(trial : Trial) -> float :

    params_cat = {
        "random_state" : 39,
        'learning_rate' : trial.suggest_loguniform('learning_rate', 0.005, 0.5), # 학습률이 0.005와 0.5 사이의 랜덤한 값으로 정해줌
        "n_estimators" : trial.suggest_int("n_estimators", 400, 1000), # 400과 1000 사이에 랜덤한 값으로 정해줌
        "max_depth" : trial.suggest_int("max_depth", 3, 10) # 3과 10 사이에 랜덤한 값으로 정해줌
  }
    
    model = CatBoostClassifier(**params_cat) # Catboost가 AutoML에서 가장 좋아서 Catboost로 했다.
    model.fit(X_partrain, y_partrain, eval_set = [(X_val, y_val)],
              early_stopping_rounds = 100, cat_features = categorical_features, verbose = False)

    cat_pred = model.predict(X_val)
    AUC = roc_auc_score(y_val, cat_pred)
    
    return AUC

In [14]:
# 이 작업을 해주면 어떤 초모수가 가장 좋은지를 얘가 판단을 해준다.
# 나는 총 100번 돌렸지만, 원래는 1000번 정도 돌리는게 좋다.

sampler = TPESampler(seed = 39)
study = optuna.create_study(
    study_name = "cat_parameter_opt",
    direction = "maximize",
    sampler = sampler)
study.optimize(objective, n_trials = 100)

[I 2022-11-26 15:22:23,676] A new study created in memory with name: cat_parameter_opt
[I 2022-11-26 15:22:31,789] Trial 0 finished with value: 0.7639889912829753 and parameters: {'learning_rate': 0.06205093309656499, 'n_estimators': 879, 'max_depth': 9}. Best is trial 0 with value: 0.7639889912829753.
[I 2022-11-26 15:22:41,024] Trial 1 finished with value: 0.7626039497317288 and parameters: {'learning_rate': 0.008771416592003579, 'n_estimators': 761, 'max_depth': 7}. Best is trial 0 with value: 0.7639889912829753.
[I 2022-11-26 15:22:46,599] Trial 2 finished with value: 0.7652447670637978 and parameters: {'learning_rate': 0.04234350615938281, 'n_estimators': 683, 'max_depth': 8}. Best is trial 2 with value: 0.7652447670637978.
[I 2022-11-26 15:22:50,371] Trial 3 finished with value: 0.7548107635059454 and parameters: {'learning_rate': 0.35505674331734294, 'n_estimators': 890, 'max_depth': 10}. Best is trial 2 with value: 0.7652447670637978.
[I 2022-11-26 15:22:53,565] Trial 4 finishe

In [15]:
# 가장 좋은 초모수를 알려준다. 

print("Best Score :", study.best_value)
print("Best trial :", study.best_trial.params)

Best Score : 0.7692706259471136
Best trial : {'learning_rate': 0.26423614085760533, 'n_estimators': 473, 'max_depth': 8}


In [16]:
# 성능이 좋아지는 과정
# 시각화

optuna.visualization.plot_optimization_history(study)

In [17]:
# 파라미터들과의 관계
# 이걸 보면 처음에 초모수 범위 정해줄때 어떤게 좋은지 내가 볼 수 있다.

optuna.visualization.plot_parallel_coordinate(study)

In [18]:
# 하이퍼파라미터 중요도
# 중요한 초모수를 파악할 수 있다.

optuna.visualization.plot_param_importances(study)

In [59]:
# 교차 검증해서 얻은 초모수를 가지고 train 데이터의 예측 불량률을 파악한다.

n_fold = 5
cv = StratifiedKFold(n_splits = n_fold, shuffle = True, random_state = 39)

cat_val = np.zeros((X_train.shape[0], 2))
cat_partrain = np.zeros((X_partrain.shape[0], 2))

print(cat_val.shape)
print(cat_partrain.shape)

for i, (i_trn, i_val) in enumerate(cv.split(X_train, y_train), 1):
    print(f'training model for CV #{i}')
    optuna_cat = CatBoostClassifier(
        random_state = 39,
        learning_rate = 0.03142344166841527, 
        n_estimators = 513, 
        max_depth = 6)

    optuna_cat.fit(X_train.loc[i_trn, :], y_train[i_trn], verbose = False, cat_features = categorical_features)

    cat_val[i_val, :] = optuna_cat.predict_proba(X_train.loc[i_val, :])
    cat_partrain += optuna_cat.predict_proba(X_partrain) / n_fold

(14095, 2)
(9866, 2)
training model for CV #1
training model for CV #2
training model for CV #3
training model for CV #4
training model for CV #5


In [60]:
# 정상률 말고 불량률만 train 데이터에 따로 붙여서 train 데이터의 변수는 총 22개로 늘어났다.

print(cat_val.shape)
print(cat_partrain.shape)

train3['model1_prob'] = cat_val[:, 1]
print(train3.shape)

(14095, 2)
(9866, 2)
(14095, 22)


## (2) model2

In [61]:
# y 값은 이제 Y_LABEL이 아닌 앞에서 구한 불량률인 model1_prob이다.

X_train2 = train3.drop(['AL', 'BA', 'Y_LABEL', 'model1_prob'], axis = 1)
y_train2 = train3['model1_prob']
print(X_train2.shape)
print(y_train2.shape)

(14095, 18)
(14095,)


In [62]:
X_partrain, X_val, y_partrain, y_val = train_test_split(X_train2, y_train2, test_size = 0.3, random_state = 39)
print(X_partrain.shape)
print(X_val.shape)
print(y_partrain.shape)
print(y_val.shape)

(9866, 18)
(4229, 18)
(9866,)
(4229,)


In [65]:
# AutoML 보니까 분류모형도 회귀모형도 그냥 Catboost가 제일 좋았다.
# 그래서 model2의 교차검증도 아래와 같이 했다.

from catboost.core import CatBoostRegressor
def objective(trial : Trial) -> float :

    params_cat = {
        "random_state" : 39,
        'learning_rate' : trial.suggest_loguniform('learning_rate', 0.005, 0.5),
        "n_estimators" : trial.suggest_int("n_estimators", 400, 1000),
        "max_depth" : trial.suggest_int("max_depth", 3, 10)
  }
    
    model = CatBoostRegressor(**params_cat)
    model.fit(X_partrain, y_partrain, eval_set = [(X_val, y_val)],
              early_stopping_rounds = 100, cat_features = categorical_features, verbose = False)

    cat_pred = model.predict(X_val)
    MAE = mean_absolute_error(y_val, cat_pred)
    
    return MAE

In [66]:
sampler = TPESampler(seed = 39)
study = optuna.create_study(
    study_name = "cat_parameter_opt",
    direction = "minimize",
    sampler = sampler)
study.optimize(objective, n_trials = 100)

[I 2022-11-26 15:59:37,943] A new study created in memory with name: cat_parameter_opt
[I 2022-11-26 15:59:43,349] Trial 0 finished with value: 0.08179879034175916 and parameters: {'learning_rate': 0.06205093309656499, 'n_estimators': 879, 'max_depth': 9}. Best is trial 0 with value: 0.08179879034175916.
[I 2022-11-26 15:59:49,847] Trial 1 finished with value: 0.08190256238982123 and parameters: {'learning_rate': 0.008771416592003579, 'n_estimators': 761, 'max_depth': 7}. Best is trial 0 with value: 0.08179879034175916.
[I 2022-11-26 15:59:54,384] Trial 2 finished with value: 0.0818997637086993 and parameters: {'learning_rate': 0.04234350615938281, 'n_estimators': 683, 'max_depth': 8}. Best is trial 0 with value: 0.08179879034175916.
[I 2022-11-26 15:59:57,630] Trial 3 finished with value: 0.0821231625427511 and parameters: {'learning_rate': 0.35505674331734294, 'n_estimators': 890, 'max_depth': 10}. Best is trial 0 with value: 0.08179879034175916.
[I 2022-11-26 16:00:00,363] Trial 4 f

KeyboardInterrupt: ignored

In [25]:
# model2의 초모수 결정
print("Best Score :", study.best_value)
print("Best trial :", study.best_trial.params)

Best Score : 0.09488268688010554
Best trial : {'learning_rate': 0.058173299448250754, 'n_estimators': 870, 'max_depth': 10}


In [26]:
# 시각화
optuna.visualization.plot_optimization_history(study)

In [27]:
# 파라미터들과의 관계
optuna.visualization.plot_parallel_coordinate(study)

In [28]:
# 하이퍼파라미터 중요도
optuna.visualization.plot_param_importances(study)

In [67]:
# test 데이터의 예측 불량률을 구한다.
# 물론 음수 값도 있고 1보다 큰 값도 생겨날 것이다.
# 이 모델은 0과 1 사이에 연속된 숫자값만 나오지는 않기 때문에 -> 회귀모형이다.

n_fold = 5
cv = KFold(n_splits = n_fold, shuffle = True, random_state = 39)

cat_val = np.zeros((X_train2.shape[0]))
cat_test = np.zeros((X_test.shape[0]))

print(cat_val.shape)
print(cat_test.shape)

for i, (i_trn, i_val) in enumerate(cv.split(X_train2, y_train2), 1):
    print(f'training model for CV #{i}')
    optuna_cat = CatBoostRegressor(
        random_state = 39,
        learning_rate = 0.01310047432090872, 
        n_estimators = 848, 
        max_depth = 9)

    optuna_cat.fit(X_train2.loc[i_trn, :], y_train2[i_trn], verbose = False, cat_features = categorical_features)

    cat_val[i_val] = optuna_cat.predict(X_train2.loc[i_val, :])
    cat_test += optuna_cat.predict(X_test) / n_fold

(14095,)
(6041,)
training model for CV #1
training model for CV #2
training model for CV #3
training model for CV #4
training model for CV #5


In [68]:
# 과연 불량을 판단하는 기준이 몇인지를
# train 데이터로 먼저 판단을 해본 것이다.
# 여기서는 0.22가 아니고 0.12일 때가 0.262664로 score(맞힌 확률)가 가장 높았다.

scores = []
for threshold in range(50) :
    threshold = threshold / 50
    pred = cat_val
    pred = np.where(pred >= threshold, 1, 0)
    score = f1_score(y_train, pred)
    scores.append(score)

temp1 = pd.DataFrame(np.linspace(0, 0.98, 50), columns = ['threshold'])
temp2 = pd.DataFrame(scores, columns = ['score'])
scores = pd.concat([temp1, temp2], axis = 1)
scores

,threshold,score
0,0.00,0.157399
1,0.02,0.159931
2,0.04,0.176315
3,0.06,0.201603
4,0.08,0.229003
5,0.10,0.246230
6,0.12,0.262664
7,0.14,0.258419
8,0.16,0.223847
9,0.18,0.182014


In [78]:
temp = np.zeros(cat_val.shape[0])

for i in range(cat_val.shape[0]) :
  if cat_val[i] >= 0.14 :
    temp[i] = 1

print(classification_report(y_train, temp))
print(confusion_matrix(y_train, temp))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92     12892
           1       0.23      0.29      0.26      1203

    accuracy                           0.86     14095
   macro avg       0.58      0.60      0.59     14095
weighted avg       0.87      0.86      0.86     14095

[[11716  1176]
 [  850   353]]


In [87]:
# 하지만 test 데이터에서 0.12로 해보니까
# 불량 값이 1000개가 넘어갔다.
# 따라서 0.12는 아니지만 그 부근이겠거니 싶었다.
# 그래서 0.15정도로 해서 500개 정도에 가깝게 나오게 했다.
answer = np.zeros(cat_test.shape[0])

for i in range(cat_test.shape[0]) :
  if cat_test[i] >= 0.152 :
    answer[i] = 1

Counter(answer)

Counter({0.0: 5546, 1.0: 495})

In [88]:
# 제출
# 그랬더니 등수가 3등으로 올라가더라
# 물론 10분만에 4등으로 내려왔다.

submission_preds = answer
submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/공모전/건설기계 오일 상태 분류 AI 경진대회/open/sample_submission.csv')
submission['Y_LABEL'] = submission_preds
submission.to_csv('/content/drive/MyDrive/Colab Notebooks/공모전/건설기계 오일 상태 분류 AI 경진대회/결과/submission_catboost6.csv', index = False)

In [89]:
cat_test = pd.DataFrame(cat_test)

In [ ]:
cat_test.to_csv('/content/drive/MyDrive/Colab Notebooks/공모전/건설기계 오일 상태 분류 AI 경진대회/prob 저장용/submission_catboost3_test_prob.csv', index = False)